<a href="https://colab.research.google.com/github/GarlandZhang/cs486_project/blob/main/hashtag_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Standard imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.16.1
import numpy as np
import math
import keras
import keras.backend as K
from keras import Model
from keras.models import load_model
from keras.layers import Embedding, Dot, Add
from keras.activations import sigmoid
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications.vgg19 import VGG19
import os
from PIL import Image
import cv2
import pickle
import tqdm

     |████████████████████████████████| 17.3MB 203kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


Collaborative Filtering

In [ ]:
project_path = '/content/drive/My Drive/AI(CS486)'
IMG_DIM = 128
IMG_CHANNELS = 3

feature_size = 1000

hashtags = []
with open(os.path.join(project_path, 'data', 'hashtags.txt')) as f:
  for line in f:
    hashtags.append(line)

csv = np.load(os.path.join(project_path, 'data', f'test_csvs.npy'), allow_pickle=True)
inputs = np.load(os.path.join(project_path, 'data', f'test_inputs.npy'), allow_pickle=True).tolist()

train_csv = np.load(os.path.join(project_path, 'data', f'train_csvs.npy'), allow_pickle=True)
train_inputs = np.load(os.path.join(project_path, 'data', f'train_inputs.npy'), allow_pickle=True).tolist()


offset = csv[0][0] 
num_imgs = csv[-1][0] - offset + 1 # n: calculated by last_row_entry_id - first_row_entry_id in test set
num_hashtags = len(hashtags) # m

inputs_map = {}
for input in inputs:
  if inputs_map.get(input[0]) == None:
    inputs_map[input[0]] = { input[1] : True }
  else:
    inputs_map[input[0]][input[1]] = True

# labels
labels = [1 for i in range(len(inputs))]

for i in range(offset, offset + num_imgs):
  for j in range(num_hashtags):
    if inputs_map.get(i) == None:
      labels.append(0)
      inputs.append([i, j])
      inputs_map[i] = { j : True }
    elif inputs_map[i].get(j) == None: # false label
      labels.append(0)
      inputs.append([i, j])
      inputs_map[i][j] = True
      
labels = np.array(labels, dtype=np.float32)

inputs = np.array(inputs)

In [ ]:
class HashtagPredictor(Model):
  def __init__(self, num_imgs, num_hashtags, feature_size, **kwargs):
    super(HashtagPredictor, self).__init__(**kwargs)
    self.num_imgs = num_imgs + 1 # + 1 for test image ..?
    self.num_hashtags = num_hashtags
    self.feature_size = feature_size
    self.img_embedding = Embedding(self.num_imgs, self.feature_size, embeddings_initializer="he_normal", )
    self.img_bias = Embedding(self.num_imgs, 1)
    self.hashtag_embedding = Embedding(self.num_hashtags, self.feature_size, embeddings_initializer="he_normal")
    self.hashtag_bias = Embedding(self.num_hashtags, 1)

  def call(self, inputs):
    img_matrix = self.img_embedding(inputs[:, 0])
    img_bias = self.img_bias(inputs[:, 0])
    hashtag_matrix = self.hashtag_embedding(inputs[:, 1])
    hashtag_bias = self.hashtag_bias(inputs[:, 1])
    raw_preds = Dot(axes=1)([img_matrix, hashtag_matrix])
    preds_with_bias = Add()([raw_preds, img_bias, hashtag_bias])
    preds = sigmoid(raw_preds)
    return preds

In [ ]:
# ## DO NOT USE THIS: this is only for training. testing is just random
# img_id = csv[125][0] # TODO: every img id in test csv
# pred_inds = np.argwhere(inputs[:, 0] == img_id).flatten()
# pred_inputs = inputs[pred_inds]
# pred_labels = labels[pred_inds]
# preds = np.array([0 if i < 0.5 else 1 for i in model.predict(x=pred_inputs).flatten()])
# print(f'Prediction: {preds}')
# print(f'Actual:     {pred_labels}')
# model.predict(x=pred_inputs).flatten()

VGG (cold start)

In [ ]:
content_layers = ['block3_conv3']

In [ ]:
vgg_weights_path = os.path.join(project_path, 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
def get_content_model():
  vgg19 = VGG19(weights=None, include_top=False)
  vgg19.trainable = False
  model_outputs = [vgg19.get_layer(layer).output for layer in content_layers]
  model = Model(inputs=vgg19.input, outputs=model_outputs)
  vgg19.load_weights(vgg_weights_path)
  return model

In [ ]:
def load_image(path):
  img = Image.open(path)
  img = np.array(img)
  img = img[:,:,:IMG_CHANNELS]
  orig_height, orig_width, _ = img.shape
  scaling_factor = IMG_DIM / max(img.shape)
  img = cv2.resize(img, (round(orig_height * scaling_factor), round(orig_width * scaling_factor)))
  new_height, new_width, _ = img.shape
  out_img = np.zeros((IMG_DIM, IMG_DIM, IMG_CHANNELS))
  out_img[:new_height, :new_width, :] = img
  out_img = np.expand_dims(out_img, axis=0)
  out_img = tf.keras.applications.vgg19.preprocess_input(out_img)
  return out_img

In [ ]:
content_model = get_content_model()

In [ ]:
def get_content(img):
  content = content_model.predict(img)
  content = np.ravel(content)
  return content

k-nearest neighbors; find closest images based on cosine similarity

In [ ]:
def cosine_sim(vec_a, vec_b):
  return np.dot(vec_a, vec_b) / (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

In [ ]:
def knn(arr, query):
  K = 3
  dists = []
  for i, img in enumerate(arr):
    cont = get_content(img)
    dist = cosine_sim(query, cont)
    dists.append((dist, i))

  dists.sort(key=lambda x: x[0], reverse=True)
  neighbors = [d[1] for d in dists[:K]]
  return neighbors

In [ ]:
def recommend_hashtags(img):
  content = get_content(img)

  # generate the new weights and set them to the test row
  neighbors = knn(imgs, content)
  all_weights = model.img_embedding.weights[0].numpy()
  new_weights = np.mean(all_weights[neighbors], axis=0)
  all_weights[num_imgs, :] = new_weights
  model.img_embedding.set_weights([all_weights])

  # predict test image based on new weights set

  test_id = num_imgs
  pred_inputs = [(test_id, i) for i in range(num_hashtags)]
  preds = np.array([0 if i < 0.5 else 1 for i in model.predict(x=pred_inputs).flatten()])
  print(f'Prediction: {preds}')
  # print(f'Actual:     {pred_labels}')
  model.predict(x=pred_inputs).flatten()

  pred_hashtags = []
  for i, prob in enumerate(model.predict(x=pred_inputs).flatten()):
    if prob >= 0.5:
      pred_hashtags.append(hashtags[i])

  return pred_hashtags

In [ ]:
def f1_score(rec_hashtags, bitmap, f1_data):
  for tag in rec_hashtags:
    if bitmap[hashtags.index(tag)] == 1:
      f1_data['TP'] += 1
    else:
      f1_data['FP'] += 1
  f1_data['FN'] += bitmap.count(1) - f1_data['TP']
  return

In [ ]:
imgs = [load_image(os.path.join(project_path, 'data', 'images', row[1] + '.jpg')) for row in tqdm.tqdm(train_csv)] # might be too large


100%|██████████| 2348/2348 [12:18<00:00,  3.18it/s]


In [ ]:
f1_list = []
for bs in [16, 64, 256]:
  for lr in [1e-2, 1e-3, 1e-4]:
    model = load_model(os.path.join(project_path, 'models', f'model_bs_{bs}_lr_{lr}_fs_1000'))
    f1_data = {'TP': 0, 'FN': 0, 'FP': 0}
    for i, row in enumerate(csv):
      img_id, img_name, labels = row
      path = os.path.join(project_path, 'data', 'images', img_name + '.jpg')
      img = load_image(path)
      rec_hashtags = recommend_hashtags(img)
      f1_score(rec_hashtags, labels, f1_data)
      print(rec_hashtags)
    f1 = f1_data['TP'] / (f1_data['TP'] + 1/2 * (f1_data['FP'] + f1_data['FN']))
    f1_list.append(f1)
    print(f'Model with batch size: {bs} and learning rate: {lr} gives f1 score: {f1}')

Prediction: [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['#love\n', '#instagood\n', '#instagram\n', '#style\n', '#me\n', '#motivation\n', '#workout\n']
Prediction: [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['#love\n', '#photooftheday\n', '#girl\n']
Prediction: [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['#beautiful\n', '#photography\n']
Prediction: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0